If Notebook 1 successfully completed then we are ready to move into Training Phase

In [ ]:
import os
os.chdir("drive/My Drive/Object_Detection/models/research")

In [ ]:
# compils the proto buffers
!protoc object_detection/protos/*.proto --python_out=.

# exports PYTHONPATH environment var with research and slim paths
os.environ['PYTHONPATH'] += ':./:./slim/'

In [ ]:
!python object_detection/model_main.py \
    --pipeline_config_path="object_detection/samples/configs/ssd_mobilenet_v2_coco.config" \
    --model_dir="training/"

# Exporting latest checkpoint for predicting Image

In [ ]:
#the location where the exported model will be saved in.
output_directory = 'fine_tuned_model/'

# goes through the model is the training/ dir and gets the last one.
# you could choose a specfic one instead of the last
lst = os.listdir("training")
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = os.path.join("training", last_model)

#exports the model specifed and inference graph
!python object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path="object_detection/samples/configs/ssd_mobilenet_v2_coco.config" \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
# Write Python3 code here 
import os 
import cv2 
import numpy as np 
import tensorflow as tf 
import sys  
  
# Import utilites 
# from utils import label_map_util
# from utils import visualization_utils as vis_util
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Grab path to current working directory 
CWD_PATH = os.getcwd() 

# Name of the directory containing the object detection module we're using 
# The path to the directory where frozen_inference_graph is stored.
MODEL_NAME = 'fine_tuned_model'    
  
# Path to frozen detection graph .pb file, which contains the model that is used 
# for object detection. 
PATH_TO_CKPT = os.path.join(CWD_PATH, MODEL_NAME, 'frozen_inference_graph.pb') 
  
# Path to label map file 
PATH_TO_LABELS = '/content/drive/My Drive/Object_Detection/data/label_map.pbtxt'
  
# Path to image 
PATH_TO_IMAGE = "/content/drive/My Drive/Object_Detection/data/images/015457551102RAB1E - 160_1.jpg" 
  
# Number of classes the object detector can identify 
NUM_CLASSES = 3
  
# Load the label map. 
# Label maps map indices to category names, so that when our convolution 
# Here we use internal utility functions, but anything that returns a 
# dictionary mapping integers to appropriate string labels would be fine 
label_map = label_map_util.load_labelmap(PATH_TO_LABELS) 
categories = label_map_util.convert_label_map_to_categories( 
        label_map, max_num_classes = NUM_CLASSES, use_display_name = True) 
category_index = label_map_util.create_category_index(categories) 
  
# Load the Tensorflow model into memory. 
detection_graph = tf.Graph() 
with detection_graph.as_default(): 
    od_graph_def = tf.GraphDef() 
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid: 
        serialized_graph = fid.read() 
        od_graph_def.ParseFromString(serialized_graph) 
        tf.import_graph_def(od_graph_def, name ='') 
  
    sess = tf.Session(graph = detection_graph) 
  
# Define input and output tensors (i.e. data) for the object detection classifier 
# Input tensor is the image 
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') 
  
# Output tensors are the detection boxes, scores, and classes 
# Each box represents a part of the image where a particular object was detected 
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0') 
  
# Each score represents level of confidence for each of the objects. 
# The score is shown on the result image, together with the class label. 
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0') 
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0') 
  
# Number of objects detected 
num_detections = detection_graph.get_tensor_by_name('num_detections:0') 
  
# Load image using OpenCV and 
# expand image dimensions to have shape: [1, None, None, 3] 
# i.e. a single-column array, where each item in the column has the pixel RGB value 
image = cv2.imread(PATH_TO_IMAGE) 
image_expanded = np.expand_dims(image, axis = 0) 
  
# Perform the actual detection by running the model with the image as input 
(boxes, scores, classes, num) = sess.run( 
    [detection_boxes, detection_scores, detection_classes, num_detections], 
    feed_dict ={image_tensor: image_expanded}) 
  
# Draw the results of the detection (aka 'visualize the results') 
  
vis_util.visualize_boxes_and_labels_on_image_array( 
    image, 
    np.squeeze(boxes), 
    np.squeeze(classes).astype(np.int32), 
    np.squeeze(scores), 
    category_index, 
    use_normalized_coordinates = True, 
    line_thickness = 8, 
    min_score_thresh = 0.60) 
print(np.squeeze(classes).astype(np.int32))

# All the results have been drawn on the image. Now display the image. 
cv2.imshow('Object detector', image) 
  
# Press any key to close the image 
cv2.waitKey(0) 
  
# Clean up 
cv2.destroyAllWindows()